In [1]:
import operator
import re
from copy import deepcopy

In [2]:
workflows: dict[str, list] = {}
parts: list[dict] = []

with open("input.txt", "rt") as f:
    workflows_str, parts_str = f.read().strip().split("\n\n")

for workflow_str in workflows_str.split("\n"):
    name, rules_str = re.match("(\w+){(.+)}", workflow_str).groups()
    rules = []
    for rule_str in rules_str.split(","):
        rule = re.match("([xmas])([<>])(\d+):(\w+)|(\w+)", rule_str).groups()
        category, sign, value, target, default = rule
        value = int(value) if value is not None else None
        rules.append((category, sign, value, target, default))
    workflows[name] = rules

for part_str in parts_str.split("\n"):
    x, m, a, s = re.match("\{x=(\d+),m=(\d+),a=(\d+),s=(\d+)\}", part_str).groups()
    parts.append({"x": int(x), "m": int(m), "a": int(a), "s": int(s)})

sign_map = {">": operator.gt, "<": operator.lt}

# Part 1

In [3]:
accepted = []

for part in parts:
    workflow = workflows["in"]
    while True:
        move_to = None

        for rule in workflow:
            category, sign, value, target, default = rule

            if default is not None:
                move_to = default
            elif sign_map[sign](part[category], value):
                move_to = target
                break

        if move_to == "R":
            break

        if move_to == "A":
            accepted.append(part)
            break

        workflow = workflows[move_to]

sum(sum(part.values()) for part in accepted)

373302

In [4]:
to_check = [
    {
        "x": {"min": 1, "max": 4000},
        "m": {"min": 1, "max": 4000},
        "a": {"min": 1, "max": 4000},
        "s": {"min": 1, "max": 4000},
        "move_to": "in",
    }
]

accepted_ranges = []

while to_check:
    parts_range = to_check.pop()

    if parts_range["move_to"] == "R":
        continue
    if parts_range["move_to"] == "A":
        accepted_ranges.append(parts_range)
        continue

    workflow = workflows[parts_range["move_to"]]
    for rule in workflow:
        cat, sign, value, target, default = rule

        if default is not None:
            parts_range["move_to"] = default
            to_check.append(parts_range)

        elif value >= parts_range[cat]["min"] and value <= parts_range[cat]["max"]:
            parts_range_split = deepcopy(parts_range)
            parts_range_split["move_to"] = target

            if sign == ">":
                parts_range_split[cat]["min"] = value + 1
                parts_range[cat]["max"] = value

            elif sign == "<":
                parts_range_split[cat]["max"] = value - 1
                parts_range[cat]["min"] = value

            to_check.append(parts_range_split)

n_accepted = 0
for parts_range in accepted_ranges:
    combinations = 1
    for cat in "xmas":
        combinations *= parts_range[cat]["max"] - parts_range[cat]["min"] + 1
    n_accepted += combinations
n_accepted

130262715574114